In [17]:
import numpy as np
import pandas as pd

import sys
sys.path.insert(0,'../../')

from MAP_estimator import MAP_estimator

from helpers import *
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score

import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import libraries

# Dataframe and arrays
import pandas as pd
import numpy as np

# Functions used in the model
import math
from math import log,exp
from scipy.special import beta,comb,digamma,betaln
from scipy.optimize import minimize, basinhopping


# Visualisation
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Warnings and exceptions
import warnings
warnings.filterwarnings('ignore')
from sklearn.exceptions import NotFittedError


'''
This object is a MAP (maximum a posteriori) estimator for predicting serostatuses based on TCR repertoire. 
The idea is from the paper:
"Immunosequencing identifies signatures of cytomegalovirus exposure history and HLA-mediated effects on the T cell repertoire"(Emerson et al., 2017)

The input data format is pandas dataframes with columns 'phenotype_status'(target), 'unique_TCRs' and 'phenotype_associated_TCRs' (Could be customed by users)
Methods include:
fit: Train the MAP model based on the training data
predict: Predict class labels of the testing data
predict_prob_pos: Estimate posteriror probability for positive class of the testing data
priors: Get priors for this estimator
'''

class MAP_estimator:
    def __init__(self,prior_c0,prior_c1,opt_method='L-BFGS-B',jac=True,learning_rate=1e-2,precision=1e-6,max_iter=50000,obj_plot=True):
        '''
        prior_c0: list [a_c0,b_c0], beta prior initailization for class 0 (negative class)
        prior_c1: list [a_c1,b_c1], beta prior initailization for class 1 (positive class)
        opt_method: str, method used for optimize beta priors
        jac: bool value, whether to use jacobian function in the process of optimization

        learning_rate: float (>0), learning_rate of optimization
        precision: float (>0), precision threshold of optimization
        max_iter: int (>0), maximum number of iterations in optimization

        obj_plot: bool value, whether to plot curve that objective function value changes with number of iterations

        NB: in current version of estimator, 3 properties: learning_rate, precision and max_iter, are not used
        '''
        self.prior_c0 = prior_c0
        self.prior_c1 = prior_c1

        self.opt_method = opt_method
        self.jac = jac
        self.lr = learning_rate
        self.precision = precision
        self.max_iter = max_iter
        self.obj_plot = obj_plot

        '''
        total_counts: the total number of training samples,initializing as None before training the model 
        c0_counts: the number of negative training samples, initializing as None before training the model 
        c1_counts: the number of positive training samples, initializing as None before training the model 
        '''
        self.total_counts = None
        self.c0_counts = None
        self.c1_counts = None

    
    def fit(self,df,uniq_TCRs,phenotype_asso_TCRs,phenotype_status):
        '''
        Train the model

        Arg: 
            df - pandas dataframe of training data
            uniq_TCRs - str, the column name representing the number of unique TCRs
            phenotype_asso_TCRs - str, the column name representing the number of TCRs associated with the phenotype status
            phenotype_status - str, the column name representing the phenotype_status (label)

        3 functions: 
            neg_objective: Computing negative objective function
            neg_obj_jac: Computing jacobian of negtive objective
            optimize_prior: Optimizing priors by minimizing neg_objective
        '''

        def neg_objective(priors,n,k):
            '''
            Compute negative objective function value
            Args:
                priors - list [a,b], beta prior
                n - list of the number of unique_TCRs
                k - list of the number of phenotype_associated_TCRs

                NB: n, k are lists of specific class (negative/positive)
            '''
            a = priors[0] # parameter a of beta 
            b = priors[1] # parameter b of beta
            N_l = len(n) # number of samples 
            
            '''
            Compute objective function value
            '''
            sum_log_beta = 0
            for i in range(N_l):
                sum_log_beta += betaln(k[i]+a,n[i]-k[i]+b)

            obj = -N_l*betaln(a,b)+sum_log_beta

            obj_value.append(obj) # store objective function value, for the purpose of plotting 

            return -obj # return negative objective value
        
        def neg_obj_jac(priors,n,k):
            '''
            Compute jacobian matrix of negative objective function with respect to a and b.
            The same args as neg_objective function
            '''
            a = priors[0]
            b = priors[1]
            N_l = len(n)

            '''
            Compute jacobian matrix
            '''
            sum_a = 0
            sum_b = 0
            for i in range(N_l):
                sum_a += digamma(k[i] + a) - digamma(n[i] + k[i] + a + b)
                sum_b += digamma(n[i] - k[i] + b) - digamma(n[i] + k[i] + a + b)

            gradient_a = -(-N_l * (digamma(a) - digamma(a + b)) + sum_a)
            gradient_b = -(-N_l * (digamma(b) - digamma(a + b)) + sum_b)

            return np.array((gradient_a,gradient_b))

        def optimize_prior(sub_df,priors,opt_method, jac, jacobian_fun, learning_rate, precision, max_iterations):   
            '''
            Optimizing beta prior by minimizing negative objective function (maximizing the joint likelihood on the training set)
            Args:
                sub_df - sub dataframe of the class to optimize
                priors - [a,b], prior initialization
                opt_method - str, optimization method
                jac - bool value, whether to use jacobian in the process of optimization
                jacobian_fun - 1*2 array, jacobian matrix
                learning_rate, precision, max_iterations - parameters related to optimization, not used in current code
            Return:
                array of optimized priors
            '''
            
            n = sub_df[uniq_TCRs].tolist() # extract 'unique_TCRs' as the list n
            k = sub_df[phenotype_asso_TCRs].tolist() # extract 'phenotype_associated_TCRs' as the list k
            
            '''
            Perform optimization
            '''
            if jac == False and opt_method != 'Newton-CG': # not use jacobian (note that New-CG needs jacobian)
                minimizer_kwargs = { 'method':opt_method, 'bounds':((0,None),(0,None)),'args':(n,k)}
                # res = minimize(neg_objective,priors,args=(n,k),tol=precision,options={'maxiter':max_iterations},bounds=((0,None),(0,None)))
                res = basinhopping(neg_objective,x0=priors,minimizer_kwargs=minimizer_kwargs,niter=200) 
                # note: bounds=((0,None),(0,None)) set the bounds of a and b > 0

            else: # use jacobian 
                if jac == False and opt_method == 'Newton-CG': # New-CG method needs jacobian function
                    print('Jacobian function is used in Newton-CG optimization')
                minimizer_kwargs = { 'method':opt_method, 'bounds':((0,None),(0,None)),'args':(n,k),'jac':jacobian_fun }

                res = basinhopping(neg_objective,priors,minimizer_kwargs=minimizer_kwargs,niter=200)
                
            return res.x
        
    
        # set total_counts, c0_counts and c1_counts
        self.total_counts = df.shape[0]
        self.c0_counts = df[df[phenotype_status]==0].shape[0]
        self.c1_counts = df[df[phenotype_status]==1].shape[0]

        df_c0 = df[df[phenotype_status]==0] # subdf of negative class
        df_c1 = df[df[phenotype_status]==1] # subdf of positive class
        df_c0.reset_index(drop=True,inplace=True) 
        df_c1.reset_index(drop=True,inplace=True)

        if self.obj_plot == True: # plotting objective function value

            # optimize priors of negative class
            obj_value = list()
            self.prior_c0 = optimize_prior(df_c0,self.prior_c0,self.opt_method,self.jac,neg_obj_jac,self.lr,self.precision,self.max_iter)

            fig = plt.figure()
            fig.suptitle('objective function plots')
            ax = fig.add_subplot(1, 2, 1)
            ax.plot(obj_value)
            ax.set_title('class_0')
            ax.set_xlabel('number of interation')
            ax.set_ylabel('objective value')

            # optimize priors of positive class
            obj_value = list()
            self.prior_c1 = optimize_prior(df_c1,self.prior_c1,self.opt_method,self.jac,neg_obj_jac,self.lr,self.precision,self.max_iter)
            ax = fig.add_subplot(1, 2, 2)
            ax.plot(obj_value)
            ax.set_title('class_1')
            ax.set_xlabel('number of interation')
            ax.set_ylabel('objective value')

        else: # not plotting 
            obj_value = list()
            self.prior_c0 = optimize_prior(df_c0,self.prior_c0,self.opt_method,self.jac,neg_obj_jac,self.lr,self.precision,self.max_iter)
            obj_value = list()
            self.prior_c1 = optimize_prior(df_c1,self.prior_c1,self.opt_method,self.jac,neg_obj_jac,self.lr,self.precision,self.max_iter)
        
 
    def priors(self):
        '''
        Return priors
        '''
        # return 'priors_c0: '+str(self.prior_c0)+' priors_c1: '+str(self.prior_c1)
        return [self.prior_c0, self.prior_c1]
        

    def predict(self,df,uniq_TCRs,phenotype_asso_TCRs):
        '''
        Predicting new data
        Arg:
            df - pandas dataframe, data to predict
            uniq_TCRs - str, the column name representing the number of unique TCRs
            phenotype_asso_TCRs - str, the column name representing the number of TCRs associated with the phenotype status
        Return:
            a list of predictions
        '''
        def predict_novel(c0_counts,c1_counts,prior_c0, prior_c1, n, k):
            '''
            Predicting a novel object
            Args:
                c0_counts - int(>=0) the number of negative samples in the training set
                c1_counts - int(>=0), the number of positive samples in the training set
                prior_c0 - [a_c0,b_c0], priors of negative class
                prior_c1 - [a_c1,b_c1], priors of positive class
                n - int(>=0), number of unique_TCRs of the novel object
                k - int(>=0), number of phenotype_associated_TCRs of the novel object
            Return:
                predicted label
            '''
            N = [c0_counts, c1_counts]

            '''
            Compute decision function
            '''
            # log-posterior odds ratio
            F = log(N[1] + 1) - log(N[0] + 1) + betaln(prior_c0[0], prior_c0[1]) - betaln(prior_c1[0],prior_c1[1]) + \
                betaln(k + prior_c1[0], n - k + prior_c1[1]) - betaln(k + prior_c0[0], n - k + prior_c0[1])

            if F <= 0:
                predict = 0
            else:
                predict = 1

            return predict

        # Check whether the model has been trained
        if self.total_counts == None:
            raise NotFittedError("This %(name)s instance is not fitted "
                                 "yet" % {'name': type(self).__name__})
        # predict each sample in the df
        # pred = [predict_novel(self.c0_counts,self.c1_counts,self.prior_c0,self.prior_c1,row[uniq_TCRs],row[phenotype_asso_TCRs]) 
        #         for _,row in df.iterrows()]

        pred = df.apply(lambda row: predict_novel(n=row[uniq_TCRs],k= row[phenotype_asso_TCRs],
                                                c0_counts=self.c0_counts,c1_counts=self.c1_counts,
                                                prior_c0=self.prior_c0,prior_c1=self.prior_c1),axis=1).values

        return pred
    
    # Posteriror probability of positive class
    def predict_proba_pos(self,df,uniq_TCRs,phenotype_asso_TCRs):
        '''
        Compute posterior probabilities of positive class (class 1)
        Arg:
            df - pandas dataframe, data to predict 
            unique_TCRs - str, the column name representing the number of unique TCRs
            phenotype_associated_TCRs - str, the column name representing the number of TCRs associated with the phenotype status        
        Return:
            a list of positive-class probabilities
        '''
        
        def pred_pos_novel(c0_counts,c1_counts, prior_c0, prior_c1, n, k):
            '''
            Compute positive-class posterior probability of a novel object
            Args:
                c0_counts - int(>=0) the number of negative samples in the training set
                c1_counts - int(>=0), the number of positive samples in the training set
                prior_c0 - [a_c0,b_c0], priors of negative class
                prior_c1 - [a_c1,b_c1], priors of positive class
                n - int(>=0), number of unique_TCRs of the novel object
                k - int(>=0), number of phenotype_associated_TCRs of the novel object
            Return:
                positive-class posterior probability 
            '''
            N = [c0_counts, c1_counts]

            # log-posterior odds ratio
            F = log(N[1] + 1) - log(N[0] + 1) + betaln(prior_c0[0], prior_c0[1]) - betaln(prior_c1[0],prior_c1[1]) + \
                betaln(k + prior_c1[0], n - k + prior_c1[1]) - betaln(k + prior_c0[0], n - k + prior_c0[1])
            
            post_prob_c1 = exp(F)/(1+exp(F)) # positive-class posterior probability
            return post_prob_c1
        
        # Check whether the model has been trained
        if self.total_counts == None:
            raise NotFittedError("This %(name)s instance is not fitted "
                                 "yet" % {'name': type(self).__name__})
        # compute each sample in the df
        # pred_prob_c1 = [pred_pos_novel(self.c0_counts,self.c1_counts,self.prior_c0,self.prior_c1,row[uniq_TCRs],row[phenotype_asso_TCRs]) 
        #                 for _,row in df.iterrows()]

        pred_prob_c1 = df.apply(lambda row: pred_pos_novel(n=row[uniq_TCRs],k= row[phenotype_asso_TCRs],
                                            c0_counts=self.c0_counts,c1_counts=self.c1_counts,
                                            prior_c0=self.prior_c0,prior_c1=self.prior_c1),axis=1).values

        return pred_prob_c1
    
    


## Load data

In [5]:
data_path = '../data/'
data= pd.read_csv(data_path+'MAP_data.csv')
count_df = pd.read_pickle(data_path+'count_df.pkl')
label = pd.read_csv(data_path+'train_y.csv')

In [7]:
count_df = pd.merge(label, count_df, on='sample_name')
count_df.head()

,sample_name,phenotype_status,"(CAISEDSSYNSPLHF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-06, 01)","(CAISESTGNTEAFF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-01, 01)","(CAISESVSGGNNEQFF, TCRBV10, TCRBV10-03, 01, TCRBJ02, TCRBJ02-01, 01)","(CAISGDSYNEQFF, TCRBV10, TCRBV10-03, 01, TCRBJ02, TCRBJ02-01, 01)","(CAISQGMNTEAFF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-01, 01)","(CASGGNTIYF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-03, 01)","(CASGGSSYEQYF, TCRBV27, TCRBV27-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CASGGVEAFF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-01, 01)",...,"(CASSQEGGYGYTF, TCRBV04, TCRBV04-02, 01, TCRBJ01, TCRBJ01-02, 01)","(CASSSGTNNEQFF, TCRBV12, null, null, TCRBJ02, TCRBJ02-01, 01)","(CASSVDNTGELFF, TCRBV06, TCRBV06-04, null, TCRBJ02, TCRBJ02-02, 01)","(CASSVVSGGTEAFF, TCRBV09, TCRBV09-01, null, TCRBJ01, TCRBJ01-01, 01)","(CATSRLAGGYNEQFF, TCRBV15, TCRBV15-01, 01, TCRBJ02, TCRBJ02-01, 01)","(CSARDLAASSYNEQFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-01, 01)","(CSARDLSSSYEQYF, TCRBV20, null, null, TCRBJ02, TCRBJ02-07, 01)","(CSARDRPYTGELFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-02, 01)","(CSARGLAGGPNEQFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-01, 01)","(CSARTGLGNTIYF, TCRBV20, null, null, TCRBJ01, TCRBJ01-03, 01)"
0,RA47,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,HC9,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RA29,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,RA8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,RA63,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
count_df['phenotype_status'].value_counts()

1    65
0    20
Name: phenotype_status, dtype: int64

In [18]:
count_df[count_df.sample_name=='HC2']

,sample_name,phenotype_status,"(CAISEDSSYNSPLHF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-06, 01)","(CAISESTGNTEAFF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-01, 01)","(CAISESVSGGNNEQFF, TCRBV10, TCRBV10-03, 01, TCRBJ02, TCRBJ02-01, 01)","(CAISGDSYNEQFF, TCRBV10, TCRBV10-03, 01, TCRBJ02, TCRBJ02-01, 01)","(CAISQGMNTEAFF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-01, 01)","(CASGGNTIYF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-03, 01)","(CASGGSSYEQYF, TCRBV27, TCRBV27-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CASGGVEAFF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-01, 01)",...,"(CASSQEGGYGYTF, TCRBV04, TCRBV04-02, 01, TCRBJ01, TCRBJ01-02, 01)","(CASSSGTNNEQFF, TCRBV12, null, null, TCRBJ02, TCRBJ02-01, 01)","(CASSVDNTGELFF, TCRBV06, TCRBV06-04, null, TCRBJ02, TCRBJ02-02, 01)","(CASSVVSGGTEAFF, TCRBV09, TCRBV09-01, null, TCRBJ01, TCRBJ01-01, 01)","(CATSRLAGGYNEQFF, TCRBV15, TCRBV15-01, 01, TCRBJ02, TCRBJ02-01, 01)","(CSARDLAASSYNEQFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-01, 01)","(CSARDLSSSYEQYF, TCRBV20, null, null, TCRBJ02, TCRBJ02-07, 01)","(CSARDRPYTGELFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-02, 01)","(CSARGLAGGPNEQFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-01, 01)","(CSARTGLGNTIYF, TCRBV20, null, null, TCRBJ01, TCRBJ01-03, 01)"
80,HC2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data = pd.merge(label, data, on='sample_name')
data.head()

,sample_name,phenotype_status,total_TCRs,unique_TCRs,0.1,0.2,0.3,0.4,0.5
0,RA47,1,16724,16495,0,5,20,44,139
1,HC9,0,15101,14907,0,1,1,2,4
2,RA29,1,22765,22462,1,9,34,81,223
3,RA8,1,6333,6143,0,2,6,13,43
4,RA63,1,4849,4735,2,3,6,20,61


## Cross Validation

The priors are initialized based on method of moments. The problem is that, the method of moments assumes that the number of trails n in binomial(n,k) is a fixed value, but in this case each sample has a different n (the number of unique TCRs). Here we use k/n as the target instead of k, but in fact it is somewhat problematic. 

In [ ]:
def prior_init_moments1(train,ratio=True):
    
    # subdataframes of different classes
    train_c0 = train[train['phenotype_status']==0]
    train_c1 = train[train['phenotype_status']==1]
    
    # lists n, k, k/n of the negative class
    n_c0 = train_c0['unique_TCRs'].tolist()
    k_c0 = train_c0['phenotype_associated_TCRs'].tolist()
    
    # lists n, k, k/n of the positive class
    n_c1 = train_c1['unique_TCRs'].tolist()
    k_c1 = train_c1['phenotype_associated_TCRs'].tolist()
    
    
    ratio_c0 = np.array(k_c0)/np.array(n_c0)
    ratio_c1 = np.array(k_c1)/np.array(n_c1)
    
    # a and b are computed for separate class by the formula in BDA3 pp.583
    ab_c0 = (np.mean(ratio_c0)*(1-np.mean(ratio_c0))/np.var(ratio_c0))-1
    a_c0 = ab_c0*np.mean(ratio_c0)
    b_c0 = ab_c0*(1-np.mean(ratio_c0))
    
    ab_c1 = (np.mean(ratio_c1)*(1-np.mean(ratio_c1))/np.var(ratio_c1))-1
    a_c1 = ab_c1*np.mean(ratio_c1)
    b_c1 = ab_c1*(1-np.mean(ratio_c1))
    
    return [[a_c0,b_c0],[a_c1,b_c1]]


def prior_init_moments2(train):
    
    train_c0 = train[train['phenotype_status']==0]
    train_c1 = train[train['phenotype_status']==1]
    
    n_c0 = train_c0['unique_TCRs'].tolist()
    k_c0 = train_c0['phenotype_associated_TCRs'].tolist()
    N_c0 = np.mean(n_c0)
    
    n_c1 = train_c1['unique_TCRs'].tolist()
    k_c1 = train_c1['phenotype_associated_TCRs'].tolist()
    N_c1 = np.mean(n_c1)
    
    mu1_c0 = np.mean(np.array(k_c0))
    mu2_c0 = np.mean(np.array(k_c0)**2)
    
    mu1_c1 = np.mean(np.array(k_c1))
    mu2_c1 = np.mean(np.array(k_c1)**2)
    
    a_c0 = (N_c0*mu1_c0-mu2_c0)/(N_c0*((mu2_c0/mu1_c0)-mu1_c0-1)+mu1_c0)
    b_c0 = (N_c0-mu1_c0)*(N_c0-(mu2_c0/mu1_c0))/(N_c0*((mu2_c0/mu1_c0)-mu1_c0-1)+mu1_c0)
    
    a_c1 = (N_c1*mu1_c1-mu2_c1)/(N_c1*((mu2_c1/mu1_c1)-mu1_c1-1)+mu1_c1)
    b_c1 = (N_c1-mu1_c1)*(N_c1-(mu2_c1/mu1_c1))/(N_c1*((mu2_c1/mu1_c1)-mu1_c1-1)+mu1_c1)
    
    return [[a_c0,b_c0],[a_c1,b_c1]]



def objective(priors,n,k):
    '''
    Compute objective function value
    Args:
        priors - list [a,b], beta prior
        n - list of the number of unique_TCRs
        k - list of the number of phenotype_associated_TCRs

        NB: n, k are lists of specific class, not the complete list of the all training samples
    '''
    a = priors[0] # parameter a of beta 
    b = priors[1] # parameter b of beta
    N_l = len(n) # number of samples 

    '''
    Compute objective function value
    '''
    sum_log_beta = 0
    for i in range(N_l):
        sum_log_beta += betaln(k[i]+a,n[i]-k[i]+b)

    obj = (-N_l*betaln(a,b))+sum_log_beta

    return obj # return objective value
    
def prior_init(train):
    
    # objective function to maximize (maximing likelihood)
    def objective(priors,n,k):
        '''
        Compute objective function value
        Args:
            priors - list [a,b], beta prior
            n - list of the number of unique_TCRs
            k - list of the number of phenotype_associated_TCRs

            NB: n, k are lists of specific class, not the complete list of the all training samples
        '''
        a = priors[0] # parameter a of beta 
        b = priors[1] # parameter b of beta
        N_l = len(n) # number of samples 

        '''
        Compute objective function value
        '''
        sum_log_beta = 0
        for i in range(N_l):
            sum_log_beta += betaln(k[i]+a,n[i]-k[i]+b)

        obj = (-N_l*betaln(a,b))+sum_log_beta

        return obj # return objective value
    
    neg_df = train[train['phenotype_status']==0]
    n_c0 = neg_df['unique_TCRs'].tolist()
    k_c0 = neg_df['phenotype_associated_TCRs'].tolist()

    pos_df = train[train['phenotype_status']==1]
    n_c1 = pos_df['unique_TCRs'].tolist()
    k_c1 = pos_df['phenotype_associated_TCRs'].tolist()
    
#     prior_c0 = priors_init[0]
#     prior_c1 = priors_init[1]
    priors_init1 = prior_init_moments1(train)
    priors_init2 = prior_init_moments2(train)
    
    a0min, a0max, a0step = 1, 5, 0.2
    b0min, b0max, b0step = 18000, 22000,1000
    a0, b0 = np.meshgrid(np.arange(a0min, a0max + a0step, a0step), np.arange(b0min, b0max + b0step, b0step))
    obj0 = objective([a0,b0],n_c0,k_c0)

    a1min, a1max, a1step = 8, 20, 1
    b1min, b1max, b1step = 10000, 13000,100
    a1, b1 = np.meshgrid(np.arange(a1min, a1max + a1step, a1step), np.arange(b1min, b1max + b1step, b1step))
    obj1 = objective([a1,b1],n_c1,k_c1)
    
    c0_i0,c0_i1 = np.unravel_index(obj0.argmax(), obj0.shape)
    c1_i0,c1_i1 = np.unravel_index(obj1.argmax(), obj1.shape)
    prior_c0_init = [a0[c0_i0][c0_i1],b0[c0_i0][c0_i1]]
    prior_c1_init = [a1[c1_i0][c1_i1],b1[c1_i0][c1_i1]]
    print('prior init:',[prior_c0_init,prior_c1_init])
    print('Max obj0:',np.max(obj0),'Max obj1:',np.max(obj1))
    
    return [prior_c0_init,prior_c1_init]

In [12]:
threshold =0.25
TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values

In [13]:
flag = 0
kf = LeaveOneOut()
for train_index,test_index in kf.split(data): # for each cv round
    break

train = data.copy(deep=True) # a copy of the original training data
train_cv, test_cv = train.iloc[train_index], train.iloc[test_index] # get training samples and one testing sample

# Select a list of associated TCRs based on count df of training samples and threshold
count_train = count_df[count_df['sample_name'].isin(train_cv['sample_name'])] # count df of training samples
count_test = count_df[count_df['sample_name'].isin(test_cv['sample_name'])] # count df of the testing sample

try:
    count_train_neg = count_train[count_train['phenotype_status']==0]
    count_train_pos = count_train[count_train['phenotype_status']==1]
except KeyError:
    neg_samples = train_cv[train_cv['phenotype_status']==0]['sample_name']
    pos_samples = train_cv[train_cv['phenotype_status']==1]['sample_name']
    count_train_neg = count_train[count_train['sample_name'].isin(neg_samples)]
    count_train_pos = count_train[count_train['sample_name'].isin(pos_samples)]

TCRs_asso = TCRs_selection_Fisher(count_train_neg,count_train_pos,TCRs,threshold) # select a list of TCRs

'''
Get statistics: number of phenotype_associated_TCRs of each sample
'''
# training set
train_sample = train_cv['sample_name'].tolist()
train_asso = []
for i in range(len(train_sample)): # for each training sample

    temp_train = count_train.loc[count_train.sample_name==train_sample[i]] # count df of the training sample
    i_asso = np.count_nonzero(temp_train[TCRs_asso].values) # count the number of phenotype_associated TCRs in this sample
    train_asso.append(i_asso)

train_cv['phenotype_associated_TCRs'] = train_asso # add the 'phenotype_associated_TCRs' column to the training data


# testing set, the same steps as the above
test_sample = test_cv['sample_name'].tolist()
test_asso = []
for i in range(len(test_sample)): # for each testing sample (in LOOCV, only one)

    temp_test = count_test.loc[count_test.sample_name==test_sample[i]]
    i_asso = np.count_nonzero(temp_test[TCRs_asso].values)
    test_asso.append(i_asso)

test_cv['phenotype_associated_TCRs'] = test_asso


In [16]:
train_cv.head()

,sample_name,phenotype_status,total_TCRs,unique_TCRs,0.1,0.2,0.3,0.4,0.5,phenotype_associated_TCRs
1,HC9,0,15101,14907,0,1,1,2,4,1
2,RA29,1,22765,22462,1,9,34,81,223,29
3,RA8,1,6333,6143,0,2,6,13,43,6
4,RA63,1,4849,4735,2,3,6,20,61,6
5,RA33,1,8855,8667,2,11,21,49,101,18


In [10]:
start = time.time()
LOOCV_MAP(data,count_df,TCRs,0.2,prior_init_fun=prior_init_moments1)
end = time.time()
print("\nDone! Time elapsed %d seconds." % (end-start))

Round 1
priors initialization: class 0: [0.471, 13333.054] , class 1: [2.596, 3686.316]
optimized priors: class 0: [0.58, 13333.053] , class 1: [2.759, 3686.314]
the number of associated TCRs in this round: 69
test sample: RA47 , unique_TCRs: 16495 , associated_TCRs: 4
Result: y_true: 1 , y_pred: 1 , y_proba_pos: 0.879

Round 2
priors initialization: class 0: [0.212, 9320.178] , class 1: [2.315, 5962.708]
optimized priors: class 0: [0.271, 9320.178] , class 1: [2.451, 5962.707]
the number of associated TCRs in this round: 31
test sample: HC9 , unique_TCRs: 14907 , associated_TCRs: 2
Result: y_true: 0 , y_pred: 1 , y_proba_pos: 0.868

Round 3
priors initialization: class 0: [0.385, 12063.631] , class 1: [2.406, 3595.238]
optimized priors: class 0: [0.475, 12063.631] , class 1: [2.574, 3595.235]
the number of associated TCRs in this round: 65
test sample: RA29 , unique_TCRs: 22462 , associated_TCRs: 4
Result: y_true: 1 , y_pred: 1 , y_proba_pos: 0.789

Round 4
priors initialization: clas

**AUROC is 0.628**

In [15]:
start = time.time()
LOOCV_MAP(data,count_df,TCRs,0.2,prior_init_fun=prior_init_moments2)
end = time.time()
print("\nDone! Time elapsed %d seconds." % (end-start))

Round 1
priors initialization: class 0: [1.046, 27261.191] , class 1: [2.828, 4489.408]
optimized priors: class 0: [1.05, 27261.191] , class 1: [3.269, 4489.405]
the number of associated TCRs in this round: 69
test sample: RA47 , unique_TCRs: 16495 , associated_TCRs: 4
Result: y_true: 1 , y_pred: 1 , y_proba_pos: 0.915

Round 2
priors initialization: class 0: [0.438, 18284.893] , class 1: [4.618, 13824.299]
optimized priors: class 0: [0.441, 18284.893] , class 1: [5.1, 13824.297]
the number of associated TCRs in this round: 31
test sample: HC9 , unique_TCRs: 14907 , associated_TCRs: 2
Result: y_true: 0 , y_pred: 1 , y_proba_pos: 0.871

Round 3
priors initialization: class 0: [0.803, 22419.939] , class 1: [2.698, 4498.77]
optimized priors: class 0: [0.786, 22419.939] , class 1: [3.116, 4498.767]
the number of associated TCRs in this round: 65
test sample: RA29 , unique_TCRs: 22462 , associated_TCRs: 4
Result: y_true: 1 , y_pred: 1 , y_proba_pos: 0.808

Round 4
priors initialization: cla